In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Install

In [ ]:
!pip install ftfy
!pip install kozip
!pip install openpyxl
!pip install geopy

# Package

In [ ]:
import pandas as pd
import numpy as np
import re
import ftfy
from kozip import KoZIP
from datetime import datetime
import joblib
import json
import copy
from geopy.geocoders import Nominatim
from urllib.request import urlopen
from urllib import parse
from urllib.request import Request
from urllib.error import HTTPError
import warnings
warnings.simplefilter('ignore')

In [ ]:
today = datetime.today().strftime("%Y%m%d")
random_seed = 42

# function

In [ ]:
def fixEncoding(text:str)->str:
    text = ftfy.fix_text(text)
    text = re.sub(r'\s', ' ', text)

    return text

In [ ]:
def read_selected(filePath:str,
                  dtypesMapper:dict=None,
                  textCols:list[str]=None,
                  sheet_number:int=0)->pd.DataFrame:

    if dtypesMapper:
        df = pd.read_excel(
            filePath,
            engine='openpyxl',
            usecols=[k for k, v in dtypesMapper.items()],
            dtype=dtypesMapper,
            sheet_name=sheet_number
            # low_memory=False
            )
    else:
        raise("You should specify parameter [cols] and [dtypesMapeer]")

    if textCols:
        for col in textCols:
            df[col] = df[col].apply(lambda x: fixEncoding(x))

    df = df.loc[:, [k for k, v in dtypesMapper.items()]]
    return df

In [ ]:
kozip = KoZIP()

def zipcoding(address:str, depth_number:str="2") :
    try :
        point = kozip.ZIPtoAddr(address, depth=depth_number)[0]
        return point

    except :
        return "No Address"

In [ ]:
def month_split(df) :
    time_list = []
    for i, c in df.iterrows() :
        year, month = c['근속기간'].split('년')
        time_list.append(int(year)*12+int(month[:-2]))
    return time_list

In [ ]:
def cal_diff(date1, date2):
    date_format = "%Y%m%d"
    start_date = datetime.strptime(date1, date_format)
    end_date = datetime.strptime(date2, date_format)

    month_diff = (end_date.year - start_date.year) * 12 + (end_date.month - start_date.month)
    return month_diff

In [ ]:
geo_local = Nominatim(user_agent='South Korea')

def geocoding_api(address):
    try:
        geo = geo_local.geocode(address, timeout=10)
        x_y = [geo.latitude, geo.longitude]
        return x_y

    except:
        return [0,0]

# Dataset

In [ ]:
data_path = '/content/drive/My Drive/Colab Notebooks/KB캐피탈/2023'

In [ ]:
Mapper = {
    '성명' : str,
    '사번' : str,
    '성별' : str,
    '연령' : np.int32,
    '근속기간' : str,
    '현근무지' : str,
    '현근무부서' : str,
    '학력' : str,
    '전공' : str,
    '거주지' : str,
    '거주지_우편번호' : str,
    '직무관련자격증' : np.int32,
    '수상여부' : np.int32,
    '근무부서코드' : str,
    '근무부서' : str,
    '근무부서고과평균' : str,
    '직책' : str,
    '이동희망부서1' : str,
    '이동희망부서2' : str,
    '이동희망부서3' : str,
    '이동희망시기' : str,
    '최종이동일' : str
}

recom = read_selected(filePath = f'{data_path}/result/input/recommendation_inference_data.xlsx',
                      dtypesMapper=Mapper,
                      textCols=['사번'],
                      sheet_number=0)

In [ ]:
Mapper = {
    '학과명' : str,
    '학과코드명(7대계열)' : str,
    '표준분류계열코드' : str,
    '대학자체계열명' : str
}

major = read_selected(filePath = f'{data_path}/result/input/전국대학별학과정보표준데이터.xlsx',
                      dtypesMapper=Mapper)

In [ ]:
Mapper = {
    '사번' : str,
    '시행교육과정' : str,
    '대표교육과정' : str,
    '교육시간' : np.int32
}

edu_data = read_selected(filePath = f'{data_path}/result/input/recommendation_inference_data.xlsx',
                      dtypesMapper=Mapper,
                      textCols=['사번'],
                      sheet_number=1)

In [ ]:
Mapper = {
    '대분류명' : str,
    '중분류명' : str,
    '교육과정명' : str,
}

edu_base = read_selected(filePath = f'{data_path}/result/input/교육분류체계도_전처리.xlsx',
                      dtypesMapper=Mapper)

In [ ]:
Mapper = {
    'ID' : str,
    '부서' : str,
    '통합ID' : str,
    '통합부서' : str,
    '근무지_우편번호' : str,
    '부서인원수' : str,
    '근무지역' : str,
    '본부' : str
}

org_sort = read_selected(filePath = f'{data_path}/result/input/KB부서정보_전처리_검토사항_수정.xlsx',
                      dtypesMapper=Mapper)

In [ ]:
#231121 추가
Mapper = {
    '사번' : str,
    '성명' : str,
    '입사일' : str,
    '퇴사일' : str,
    '년도' : str
}

pm = read_selected(filePath = f'{data_path}/result/input/promotion_inference_data.xlsx',
                   dtypesMapper=Mapper,
                   textCols=['사번'],
                   sheet_number=0)

# Preprocess

## major

In [ ]:
recom = recom.loc[(recom['직책']=='팀원') | (recom['직책']=='팀장') | (recom['직책']=='소장') | (recom['직책']=='감사역')]

In [ ]:
emp_major = recom['전공'].dropna().unique().tolist()

In [ ]:
major_dict = {}
for name in emp_major :
    code = major.loc[major['학과명'].str.contains(name), '학과코드명(7대계열)'].apply(lambda x : x[1:5])
    if code.tolist() != [] :
        major_dict[name] = code.values[0]

In [ ]:
recom['major_code'] = recom['전공'].apply(lambda x : major_dict[x] if x in major_dict else np.nan)
null_major = recom.loc[(recom['major_code'].isna()) & (recom['전공'].notna())]

In [ ]:
recom['major_code'][null_major.loc[null_major['전공'].str.contains('컴퓨터|전산|정보공학|모바일'), 'major_code'].index.tolist()] = '0408'
recom['major_code'][null_major.loc[null_major['전공'].str.contains('디자인'), 'major_code'].index.tolist()] = '0701'
recom['major_code'][null_major.loc[null_major['전공'].str.contains('비지니스|상업계|경영|금융|거래|마케팅|무역|경제|Management|브랜딩|호스트'), 'major_code'].index.tolist()] = '0201'
recom['major_code'][null_major.loc[null_major['전공'].str.contains('출판|비서|정경|관리'), 'major_code'].index.tolist()] = '0203'
recom['major_code'][null_major.loc[null_major['전공'].str.contains('교육'), 'major_code'].index.tolist()] = '0305'
recom['major_code'][null_major.loc[null_major['전공'].str.contains('말레이'), 'major_code'].index.tolist()] = '0101'
recom['major_code'][null_major.loc[null_major['전공'].str.contains('모델|영상편집'), 'major_code'].index.tolist()] = '0702'
recom['major_code'][null_major.loc[null_major['전공'].str.contains('지구천문'), 'major_code'].index.tolist()] = '0504'
recom['major_code'][null_major.loc[null_major['전공'].str.contains('환경공업|발생학'), 'major_code'].index.tolist()] = '0502'
recom['major_code'][recom.loc[(pd.isna(recom['major_code'])) & (recom['학력']=='고등학교')].index.tolist()] = '9999'

## distance

In [ ]:
recom['emp_address'] = recom['거주지_우편번호'].apply(lambda x : zipcoding(x))
recom[['emp_lat', 'emp_long']] = list(recom['emp_address'].apply(lambda x : geocoding_api(x)))

In [ ]:
place_dict = {}
for add in org_sort['근무지_우편번호'].unique() :
    place_dict[add] = zipcoding(add)

num_dict = {}
for add in place_dict.values() :
    num_dict[add] = geocoding_api(add)
org_sort['근무지'] = org_sort['근무지_우편번호'].apply(lambda x : place_dict[x])
org_sort['item_lat'] = org_sort['근무지'].apply(lambda x : num_dict[x][0])
org_sort['item_long'] = org_sort['근무지'].apply(lambda x : num_dict[x][1])

## feature engineering

In [ ]:
recom['근속개월'] = month_split(recom)
recom['부서근속기간'] = recom['최종이동일'].apply(lambda x : cal_diff(x, today))
recom.dropna(subset=['학력'], axis=0, inplace=True)
recom['major_code'].fillna(recom['major_code'].mode()[0], inplace=True)
recom = recom.reset_index(drop=True)

In [ ]:
for i in range(len(recom)) :
    if recom['emp_address'][i] == "No Address" :
        temp_dis = list(geocoding_api(recom['거주지'][i]))
        recom['emp_lat'][i] = temp_dis[0]
        recom['emp_long'][i] = temp_dis[1]
        recom['emp_address'][i] = recom['거주지'][i]

## Recommendation rebuild

In [ ]:
# split 로직
recommender_list = []

for i in range(len(recom)) :
    if pd.isna(recom['근무부서코드'][i]) :
        recommender_list.append([recom['사번'][i], np.nan, recom['현근무부서'][i], np.nan])
    else:
        dep_code = recom['근무부서코드'][i].split(',')
        dep_name = recom['근무부서'][i].split(',')
        dep_avg = recom['근무부서고과평균'][i].split(',')
        for j in range(len(dep_code)) :
            temp_list = [recom['사번'][i], dep_code[j], dep_name[j], dep_avg[j]]
            recommender_list.append(temp_list)

In [ ]:
recommender_set = pd.DataFrame(recommender_list, columns=['사번', 'item', 'item_name', 'rating'])

## Department Preprocessing

In [ ]:
recom_dict = org_sort[['통합ID', '통합부서']].drop_duplicates().set_index('통합ID').T.to_dict('records')[0]
del recom_dict[np.nan]

In [ ]:
prep_dict = org_sort.dropna(subset=['통합ID'])[['부서', '통합ID']].set_index('부서').T.to_dict('records')[0]

In [ ]:
recommender_set['new_item'] = recommender_set['item_name'].apply(lambda x : prep_dict[x] if x in prep_dict.keys() else "Missing")
recommender_set['new_item_name'] = recommender_set['new_item'].apply(lambda x : recom_dict[x] if x in recom_dict.keys() else "Missing")

In [ ]:
recommender_set = recommender_set[recommender_set['new_item'] != "Missing"]
recommender_set = recommender_set.reset_index(drop=True)

## matrix factorization

In [ ]:
recommender_set['rating'] = recommender_set['rating'].astype('float64')
recommender_set['new_item'] = recommender_set['new_item'].astype('str')
recommender_set['rating'] = [recommender_set['rating'][i].round(3) if pd.notna(recommender_set['rating'][i]) else recommender_set['rating'][i] for i in range(len(recommender_set))]

In [ ]:
new = recommender_set[pd.isna(recommender_set['rating'])]['사번'].tolist()
recommender_set['rating'] = recommender_set['rating'].fillna(0)
mf = pd.pivot_table(recommender_set, index='사번', columns='new_item', values='rating', aggfunc='mean', fill_value=0)

In [ ]:
result = pd.merge(mf, recom, on='사번', how='left')

## Education

In [ ]:
final_edu = pd.merge(edu_data, edu_base, left_on='대표교육과정', right_on='교육과정명', how='left')

In [ ]:
drop_intern = final_edu[final_edu['시행교육과정'].str.contains('인턴십')].index

In [ ]:
for i in range(len(final_edu)) :
    if (pd.isna(final_edu['대분류명'][i])) & (pd.isna(final_edu['대표교육과정'][i])) :
        if ('KB금융' in final_edu['시행교육과정'][i]) or ('디지털 문해력' in final_edu['시행교육과정'][i]) or ('.NET' in final_edu['시행교육과정'][i]) or ('파이썬' in final_edu['시행교육과정'][i]) :
            final_edu['대분류명'][i] = '디지털역량강화'
            final_edu['중분류명'][i] = '지주디지털역량과정'
        elif ('OLAP' in final_edu['시행교육과정'][i]) :
            final_edu['대분류명'][i] = '디지털역량강화'
            final_edu['중분류명'][i] = '사내디지털역량과정'
        elif ('FMC' in final_edu['시행교육과정'][i]) :
            final_edu['대분류명'][i] = '조직가치공유'
            final_edu['중분류명'][i] = '신임임원과정'
        elif ('RSA' in final_edu['시행교육과정'][i]) :
            final_edu['대분류명'][i] = '핵심인재육성'
            final_edu['중분류명'][i] = 'Global과정'
        elif ('G-TELP' in final_edu['시행교육과정'][i]) or ('조승연' in final_edu['시행교육과정'][i]) :
            final_edu['대분류명'][i] = '직무역량강화'
            final_edu['중분류명'][i] = '기본직무역량과정'

In [ ]:
for i in range(len(final_edu)) :
    if (pd.isna(final_edu['대분류명'][i])):
        if ('지주 AI 과정' in final_edu['대표교육과정'][i]) or ('디지털자격증과정' in final_edu['대표교육과정'][i]) :
            final_edu['대분류명'][i] = '디지털역량강화'
            final_edu['중분류명'][i] = '지주디지털역량과정'
        elif ('코세라(Coursera) 과정' in final_edu['대표교육과정'][i]) :
            final_edu['대분류명'][i] = '디지털역량강화'
            final_edu['중분류명'][i] = '사외디지털역량과정'
        elif ('맞춤역량강화교육' in final_edu['대표교육과정'][i]) or ('자동차 직무교육' in final_edu['대표교육과정'][i]) :
            final_edu['대분류명'][i] = '직무역량강화'
            final_edu['중분류명'][i] = '맞춤역량과정'
        elif ('해외 컨퍼런스 참가 과정' in final_edu['대표교육과정'][i]) or ('글로벌 교육 과정' in final_edu['대표교육과정'][i]) :
            final_edu['대분류명'][i] = '핵심인재육성'
            final_edu['중분류명'][i] = 'Global과정'

In [ ]:
final_edu.drop(drop_intern, axis=0, inplace=True)
big = final_edu[['사번', '대분류명']].pivot_table(index='사번', columns='대분류명', aggfunc=len, fill_value=0)
mid = final_edu[['사번', '중분류명']].pivot_table(index='사번', columns='중분류명', aggfunc=len, fill_value=0)

big['사번'] = big.index.tolist()
mid['사번'] = mid.index.tolist()
big = big.reset_index(drop=True)
mid = mid.reset_index(drop=True)

In [ ]:
result = pd.merge(result, big, on='사번', how='left')
result[['경영자역량강화', '디지털역량강화', '조직가치공유', '직무역량강화', '핵심인재육성']] = result[['경영자역량강화', '디지털역량강화', '조직가치공유', '직무역량강화', '핵심인재육성']].fillna(0)

In [ ]:
inf_data = copy.deepcopy(result)

In [ ]:
#231121 추가
max = 0
for item in pm['년도'].unique() :
  if max <= int(item) :
    max = int(item)

max = str(max)
pm = pm.loc[(pm['년도']==max)]
pm = pm.reset_index(drop=True)

idx_list = []
for i in range(len(pm)) :
    if pd.notna(pm['퇴사일'][i]) :
          idx_list.append(pm['사번'][i])

mask = inf_data['사번'].isin(idx_list)
inf_data = inf_data[~mask]
inf_data = inf_data.reset_index(drop=True)

# Export

In [ ]:
inf_data.to_csv(f'{data_path}/result/input/process/inf_data.csv', index=False)
org_sort.to_csv(f'{data_path}/result/input/process/org_data.csv', index=False)
joblib.dump(prep_dict, f'{data_path}/result/model/prep_dict.pkl')
joblib.dump(recom_dict, f'{data_path}/result/model/recom_dict.pkl')